In [1]:
import h5py
import numpy as np
import math
import glob
import os
import umap
import hdbscan
import seaborn as sns
import pandas as pd
import pickle
import openpyxl
import cv2
from scipy import ndimage as ndi

from functools import reduce
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.optimize import curve_fit
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import sklearn.cluster as cluster
from moviepy.editor import *
import imageio

/Users/kaavya/anaconda3/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/kaavya/anaconda3/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/kaavya/anaconda3/lib/python3.11/site-packages/umap/distances.py:1086: NumbaDeprecatio

In [2]:
ranking_df = pd.read_excel('/Users/kaavya/Downloads/combined_final_elo_rating_df.xlsx')
ranking_df

,Unnamed: 0,cage,cage_num_of_subject,cohort,home_cage_observation_final_elo_rating,home_cage_observation_rank,reward_competition_final_elo_rating,reward_competition_rank,strain,subject_id,tube_test_final_elo_rating,tube_test_rank,urine_marking_final_elo_rating,urine_marking_rank
0,24,NaN,1,pilot_1,NaN,NaN,1088.8,1,C57,1.1,1062.2,2,1010.3,2
1,25,NaN,1,pilot_1,NaN,NaN,1067.1,2,C57,1.2,927.1,3,1000.8,3
2,26,NaN,1,pilot_1,NaN,NaN,904.3,4,C57,1.3,775.7,4,972.0,4
3,27,NaN,1,pilot_1,NaN,NaN,939.8,3,C57,1.4,1235.0,1,1016.9,1
4,28,NaN,2,pilot_1,NaN,NaN,1061.3,1,C57,2.1,1122.6,2,1037.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,59,NaN,5,pilot_3,940.6,3.0,963.0,3,CD1,5.4,1165.3,1,1019.7,1
60,60,NaN,6,pilot_3,1000.0,NaN,974.5,3,CD1,6.1,1137.7,1,980.6,3
61,61,NaN,6,pilot_3,1000.0,NaN,751.8,4,CD1,6.2,1021.2,2,980.6,3
62,62,NaN,6,pilot_3,1000.0,NaN,1244.4,1,CD1,6.3,1006.6,3,1028.8,1


In [3]:
df

def ranking_difference(filtered_ranking_df, tone_times_df, id1, id2, ranking_col, file_name, tone_rd_col):
    ranking1 = filtered_ranking_df.loc[filtered_ranking_df['subject_id'] == id1, ranking_col].values[0]
    ranking2 = filtered_ranking_df.loc[filtered_ranking_df['subject_id'] == id2, ranking_col].values[0]
    difference = ranking1 - ranking2
    tone_times_df.loc[file_name, tone_rd_col] = difference

    
id1 = 1.3
id2 = 1.4
result = ranking_difference(df, id1, id2, 'urine_marking_rank')
print(result)



3


In [9]:
filtered_ranking_df = df[df['cohort'] == 'pilot_3']
filtered_ranking_df

,Unnamed: 0,cage,cage_num_of_subject,cohort,home_cage_observation_final_elo_rating,home_cage_observation_rank,reward_competition_final_elo_rating,reward_competition_rank,strain,subject_id,tube_test_final_elo_rating,tube_test_rank,urine_marking_final_elo_rating,urine_marking_rank
40,40,NaN,1,pilot_3,1000.0,NaN,1163.2,1,C57,1.1,1165.3,1,1009.7,2
41,41,NaN,1,pilot_3,1000.0,NaN,959.9,3,C57,1.2,1054.5,2,1019.1,1
42,42,NaN,1,pilot_3,1000.0,NaN,1018.4,2,C57,1.3,882.3,4,980.9,4
43,43,NaN,1,pilot_3,1000.0,NaN,858.5,4,C57,1.4,897.9,3,990.3,3
44,44,NaN,2,pilot_3,1169.1,1.0,972.6,3,C57,2.1,1127.8,1,1009.4,2
45,45,NaN,2,pilot_3,940.4,3.0,1110.4,1,C57,2.2,965.6,3,970.6,4
46,46,NaN,2,pilot_3,952.3,2.0,1031.1,2,C57,2.3,1072.9,2,1000.6,3
47,47,NaN,2,pilot_3,938.2,4.0,885.9,4,C57,2.4,833.7,4,1019.4,1
48,48,NaN,3,pilot_3,1000.0,NaN,910.9,4,C57,3.1,847.8,4,1009.4,2
49,49,NaN,3,pilot_3,1000.0,NaN,1087.7,1,C57,3.2,1024.4,2,989.4,4


In [10]:
def ranking_difference(filtered_ranking_df, id1, id2, ranking_col):
    ranking1 = filtered_ranking_df.loc[filtered_ranking_df['subject_id'] == id1, ranking_col].values[0]
    ranking2 = filtered_ranking_df.loc[filtered_ranking_df['subject_id'] == id2, ranking_col].values[0]
    difference = ranking1 - ranking2
    return difference
id1 = 1.1
id2 = 1.2
result = ranking_difference(filtered_ranking_df, id1, id2, 'urine_marking_rank')
print(result)

1
